## <font color='green'>Setup 1</font>: Load Libraries

In [1]:
import sys
root_dir = "../"
sys.path.append(root_dir)

from test_bad_word import *
from utility import *

import time
import numpy as np
import pandas as pd
#pd.options.display.max_columns = None
#pd.options.display.mpl_style = 'default'
import tensorflow as tf

import re
import os
import sys
import random
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import sparse

from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix
    
stemmer = SnowballStemmer('english')

## <font color='green'>Setup 2</font>: Load Datasets

In [2]:
bad_word_1  = [line.rstrip('\n') for line in open('../wordlist/google_bad_word.txt')]
#bad_word_2  = [line.rstrip('\n') for line in open('handcrafted_badword.txt')]
bad_word= set(bad_word_1  + test_bad_word) 

In [3]:
df_train = pd.read_csv('data/RNN_train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('data/RNN_test.csv', encoding="ISO-8859-1")
df_val = pd.read_csv('data/RNN_val.csv', encoding="ISO-8859-1")

num_train = df_train.shape[0]
num_test = df_test.shape[0]
num_val = df_val.shape[0]

In [4]:
def data_transform(file):
    data = []
    length = len(file)
    sentences = file['Comment constructed'].values
    insults = file['Insult'].values
    for i in range(length):
        current_sentences = sentences[i]
        current_insult = insults[i]
        instance = {"sentences": eval(current_sentences), "insult": current_insult}
        data.append(instance)
    return data

train = data_transform(df_train)
test = data_transform(df_test)
val = data_transform(df_val)

In [5]:
train = train + test
test = val
num_train = num_train + num_test
num_test = num_val

df_train = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_test = df_val

In [6]:
num_train

6594

## <font color='green'>Setup 3</font>: Pipeline Datasets

* comment length: number of setences in a comment
* sentence length: number of words in a sentene

In [7]:
max_comment_len = 25
max_sentence_len = 25

In [8]:
def pipeline(data, vocab = None, max_comment_len = 17, max_sentence_len_ = None):
    is_ext_vocab = True
    if vocab is None:
        is_ext_vocab = False
        vocab = {'<PAD>': 0, '<OOV>': 1}

    max_sentence_len = -1
    data_sentences = []
    data_insults = []
    for instance in data:
        sents = []
        for sentence_id, sentence in enumerate(instance['sentences']):
            if sentence_id <= max_comment_len:
                sent = []
                tokenized = sentence.split(' ')
                for token in tokenized:
                    token = token.lower()
                    if not is_ext_vocab and token not in vocab:
                        vocab[token] = len(vocab)
                    if token not in vocab:
                        token_id = vocab['<OOV>']
                    else:
                        token_id = vocab[token]
                    sent.append(token_id)
                if len(sent) > max_sentence_len:
                    max_sentence_len = len(sent)
                sents.append(sent)
        data_sentences.append(sents)
        data_insults.append(instance['insult'])
        
    if max_sentence_len_ is not None:
        max_sentence_len = max_sentence_len_
    out_sentences = np.full([len(data_sentences), max_comment_len, max_sentence_len], vocab['<PAD>'], dtype=np.int32)

    for i, elem in enumerate(data_sentences):
        for j, sent in enumerate(elem):
            if j < max_comment_len:
                if len(sent) <= max_sentence_len:
                    out_sentences[i, j, 0:len(sent)] = sent
                else:
                    out_sentences[i, j, 0: max_sentence_len] = sent[:max_sentence_len]

    return out_sentences, np.array(data_insults), vocab

In [9]:
train_sentences, train_insults, vocab = pipeline(train, max_comment_len = max_comment_len, max_sentence_len_ = max_sentence_len)

test_sentences, test_insults, vocab = pipeline(test, vocab, max_comment_len, max_sentence_len)

In [10]:
train_sentences.shape, test_sentences.shape,

((6594, 25, 25), (2235, 25, 25))

In [11]:
from scipy.stats import threshold

def sentence_length_filler(sentence_length, max_comment_len, max_sentence_len):
    n = sentence_length.shape[0]
    out_array = np.full([n, max_comment_len], 0, dtype=np.int32)
    for i,sen_len in enumerate(sentence_length):
        sen_len = eval(sen_len)
        out_array[i,0:len(sen_len)] = sen_len[:max_comment_len]
    return threshold(out_array, threshmax = max_sentence_len)

def comment_length_filler(original_comment_length, max_comment_len):
    comment_length = []
    for i in original_comment_length:
        if i > max_comment_len:
            comment_length.append(15)
        else:
            comment_length.append(i)
    return np.array(comment_length)

In [12]:
train_comment_length = comment_length_filler(df_train['comment length'].values, max_comment_len)
train_sentences_length = df_train['sentences length'].values
train_sentences_length = sentence_length_filler(train_sentences_length, max_comment_len, max_sentence_len)

test_comment_length = comment_length_filler(df_test['comment length'].values, max_comment_len)
test_sentences_length = df_test['sentences length'].values
test_sentences_length = sentence_length_filler(test_sentences_length, max_comment_len, max_sentence_len)

In [13]:
train_comment_length_index = np.ones((num_train, max_comment_len))
test_comment_length_index = np.ones((num_test, max_comment_len))

for i,index in enumerate(train_comment_length):
    train_comment_length_index[i][index:] = 0
    
for i,index in enumerate(test_comment_length):
    test_comment_length_index[i][index:] = 0

In [14]:
train_comment_length

array([ 1,  2,  2, ...,  1,  6, 14], dtype=int64)

In [15]:
train_sentences_length

array([[ 4,  0,  0, ...,  0,  0,  0],
       [ 7,  9,  0, ...,  0,  0,  0],
       [15,  0,  0, ...,  0,  0,  0],
       ..., 
       [ 7,  0,  0, ...,  0,  0,  0],
       [19,  6, 12, ...,  0,  0,  0],
       [13, 25, 14, ...,  0,  0,  0]])

## <font color='green'>Setup 4</font>: Word2Vec

In [16]:
import collections
import operator

glove_size = 100

word_dict= collections.defaultdict(list)
vocab_keys = vocab.keys()

#file= open('word2vec/glove.6B.%sd.txt' % glove_size, 'r', encoding='utf-8')
file= open('word2vec/glove.twitter.27B.%sd.txt' % glove_size, 'r', encoding='utf-8')
#file= open('word2vec/glove.840B.300d.txt', 'r', encoding='utf-8')
for line in file:
    line = line.rstrip().split(' ')
    if line[0] in vocab_keys:
        word_dict[line[0]]=[float(i) for i in line[1:]]
    
word_dict=dict(word_dict)

In [17]:
sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))

embedding_list=[]
#OOV_vector = [random.uniform(-1, 1) for i in range(glove_size)]
OOV_vector  = np.mean(list(word_dict.values()),axis=0)

for item in sorted_vocab:
    if item[0]== '<PAD>':
        embedding_list.append(np.array([0 for i in range(glove_size)], dtype='f'))
    elif item[0] =='_cr_':
        embedding_list.append(word_dict['fuck'])
    elif item[0] in word_dict:
        embedding_list.append(word_dict[item[0]])
    else:
        if item[0] in bad_word:
            embedding_list.append(word_dict['fuck']) 
        else:
            embedding_list.append(OOV_vector)
        #embedding_list.append('unseen')
       
W = np.array(embedding_list)
#print("unseen ratio:", embedding_list.count('unseen')/len(vocab))

In [18]:
normal_index = df_train[df_train['Insult']==0].index.tolist()
insult_index = df_train[df_train['Insult']==1].index.tolist()

## <font color='green'>Setup 5</font>: Construct RNN:

### tricks which have positive effect:
* pre-trained word2vec (✓,  trainable: False > True for better generalisation)
* different max sentence/comment length ( max_comment_len: 17-->5 & max_sent_len: 50-->25 ✓)
* recheck the pipeline (lowercase tansformation ✓)

### tricks which have no obvious effect:
* regularisation: dropout/L2 
* weighted loss (insignificant)
* sentence/paragraph pooling: mean pooling

### tricks which have negative effect : 
* attention on sentence/word (×)
* downsampling mini batch training (×)

### to do list:
* self trained word2vec ?

In [19]:
### MODEL PARAMETERS ###

max_comment_len = train_sentences.shape[1]
max_sen_len = train_sentences.shape[2]
vocab_size = len(vocab)
word2vec_size = glove_size
sentence_hidden_size = 100
comment_hidden_size = 100
target_size = 2

In [20]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

In [21]:
def attention(inputs, attention_size, time_major=False):

    if isinstance(inputs, tuple):
        # In case of Bi-RNN, concatenate the forward and the backward RNN outputs.
        inputs = tf.concat(inputs, 2)

    if time_major:
        # (T,B,D) => (B,T,D)
        inputs = tf.array_ops.transpose(inputs, [1, 0, 2])

    inputs_shape = inputs.shape
    sequence_length = inputs_shape[1].value  # the length of sequences processed in the antecedent RNN layer
    hidden_size = inputs_shape[2].value  # hidden size of the RNN layer

    # Attention mechanism
    W_omega = tf.Variable(tf.random_normal([hidden_size, attention_size], stddev=0.1))
    b_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    u_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))

    v = tf.tanh(tf.matmul(tf.reshape(inputs, [-1, hidden_size]), W_omega) + tf.reshape(b_omega, [1, -1]))
    vu = tf.matmul(v, tf.reshape(u_omega, [-1, 1]))
    exps = tf.multiply(tf.reshape(tf.exp(vu), [-1, sequence_length]),comment_length_index)
    alphas = exps / tf.reshape(tf.reduce_sum(exps, 1), [-1, 1])

    # Output of Bi-RNN is reduced with attention vector
    output = tf.reduce_sum(inputs * tf.reshape(alphas, [-1, sequence_length, 1]), 1)
    return output,alphas 

In [25]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
sess = tf.InteractiveSession()

sentences = tf.placeholder(tf.int64, [None, None, None], "sentences")                      # [batch_size x max_comment_len x max_sen_len]
sentences_length = tf.placeholder(tf.int64, [None, None], "sentences_length")              # [batch_size x max_comment_len]
comment_length = tf.placeholder(tf.int64, [None], "comment_length")                        # [batch_size]
comment_length_index = tf.placeholder(tf.float32, [None, max_comment_len], "comment_length") # [batch_size x max_comment_len]   
insult = tf.placeholder(tf.int64, [None], "insult")                                        # [batch_size]
keep_prob = tf.placeholder(tf.float32, (),  'keep_prob')
learning_rate = tf.placeholder(tf.float32, (),  'learning_rate')

batch_size = tf.shape(sentences)[0]

# max_comment_len x [batch_size x max_sen_len]
sentences_lst = [tf.reshape(x, [batch_size, -1]) for x in tf.split(sentences, max_comment_len, 1)] 
sentences_length_lst = [tf.reshape(x,[-1]) for x in tf.split(sentences_length, max_comment_len, 1)] 

initializer = tf.random_uniform_initializer(-0.1, 0.1)
embeddings = tf.get_variable("W", [vocab_size, word2vec_size], initializer=initializer, trainable= True)
embeddings = embeddings.assign(W) 

# max_comment_len x [batch_size x max_sen_len x word2vec_size]
sentences_embedded = [tf.nn.embedding_lookup(embeddings, sentence)  
                          for sentence in sentences_lst] 

# split sentences_embedded into n parts, name as sentence0, sentence1, etc
for i in range(max_comment_len):
    globals()['sentences_embedded_%s' % i] = sentences_embedded[i]     # [batch_size x max_sen_len x word2vec_size]
    globals()['sentences_length_%s' % i] = sentences_length_lst[i]

### ------------------------------------------------------------------------------------------------------- ### 
### ---------------------------------------- sentence encoders -------------------------------------------  ###
### ------------------------------------------------------------------------------------------------------- ### 

lstm_sentence_cell = tf.contrib.rnn.GRUCell(sentence_hidden_size)
lstm_sentence_cell = tf.contrib.rnn.DropoutWrapper(lstm_sentence_cell)
with tf.variable_scope("sentence_encoder") as varscope: 
    _, sentence_0 = tf.nn.dynamic_rnn(lstm_sentence_cell, sentences_embedded_0,\
                                                  sequence_length = sentences_length_0, dtype=tf.float32)        

    for i in range(1, max_comment_len):
        varscope.reuse_variables()
        _, globals()['sentence_%s' % i]  = tf.nn.dynamic_rnn(lstm_sentence_cell, globals()['sentences_embedded_%s' % i],\
                                                    sequence_length = globals()['sentences_length_%s' % i], dtype=tf.float32) 


        
        
sentence_vectors = tf.stack([globals()['sentence_%s' % i]   for i in range(max_comment_len)], axis=1)

### ------------------------------------------------------------------------------------------------------- ### 
### ---------------------------------------- comment encoders --------------------------------------------  ###
### ------------------------------------------------------------------------------------------------------- ### 

lstm_comment_cell_fw = tf.contrib.rnn.GRUCell(comment_hidden_size)
lstm_comment_cell_bw = tf.contrib.rnn.GRUCell(comment_hidden_size)
lstm_comment_cell_fw = tf.contrib.rnn.DropoutWrapper(lstm_comment_cell_fw, keep_prob, keep_prob)
lstm_comment_cell_bw = tf.contrib.rnn.DropoutWrapper(lstm_comment_cell_bw, keep_prob, keep_prob)
with tf.variable_scope("comment_encoder") as varscope: 
    comment_vectors_all, comment_vectors = tf.nn.bidirectional_dynamic_rnn(lstm_comment_cell_fw,lstm_comment_cell_bw, \
                                                            sentence_vectors,sequence_length = comment_length, dtype=tf.float32)            

    
comment_all_state = tf.concat(comment_vectors_all, axis=2)
#comment_vectors = tf.div(tf.reduce_sum(comment_all_state, 1), tf.stack([tf.cast(comment_length, tf.float32)],1))    
#comment_vectors = tf.concat(comment_vectors, axis =1)   
 

comment_vectors,alphas = attention(comment_vectors_all, 200)
### ------------------------------------------------------------------------------------------------------- ### 
### ---------------------------------------- loss & prediction -------------------------------------------  ###
### ------------------------------------------------------------------------------------------------------- ###     

comment_vectors = tf.layers.batch_normalization(comment_vectors)

h = tf.contrib.layers.linear(comment_vectors, 100, activation_fn = tf.nn.relu)
h = tf.layers.batch_normalization(h)
logits = tf.contrib.layers.linear(h, target_size)
probability = tf.nn.softmax(logits)
predict = tf.argmax(probability, axis=1)

L2 = tf.add_n([ tf.nn.l2_loss(v) for v in tf.trainable_variables()]) 

loss = tf.reduce_sum(tf.losses.sparse_softmax_cross_entropy(logits = logits, labels = insult, weights = 1 ))
                                                                                          ### weights = insult*2 + 1
'''
logits = tf.contrib.layers.fully_connected(comment_vectors, 1)
labels = tf.cast(tf.stack([insult], axis=1), tf.float32)
loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits, labels = labels))    
probability = tf.sigmoid(logits)   
predict = tf.argmax(probability, axis=1)
'''
opt_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
print('graph created')

graph created


In [26]:
BATCH_SIZE = 100
split_num = 50
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    n = train_sentences.shape[0]
    print('hyperparameters: ', 'max_comment_len=', max_comment_len, ', max_sentence_len=', max_sentence_len, \
          ', sentence_hidden_size=',sentence_hidden_size, ', comment_hidden_size=', comment_hidden_size , \
          ', word2vec_size=', word2vec_size)
    
    train_sentences_splited = list(split(train_sentences, split_num))
    train_insults_splited = list(split(train_insults, split_num))
    train_comment_length_splited = list(split(train_comment_length, split_num))
    train_comment_length_index_splited = list(split(train_comment_length_index, split_num))
    train_sentences_length_splited = list(split(train_sentences_length, split_num))
    
    test_sentences_splited = list(split(test_sentences, split_num))
    test_insults_splited = list(split(test_insults, split_num))
    test_comment_length_splited = list(split(test_comment_length, split_num))
    test_comment_length_index_splited = list(split(test_comment_length_index, split_num))
    test_sentences_length_splited = list(split(test_sentences_length, split_num))
    for epoch in range(15):
        print('----- Epoch', epoch + 1, '-----')
        total_loss = 0
        learning_rate_ = 0.005 if epoch < 5 else 0.001
        t_0 = time.time()
            
        for i in range(n // BATCH_SIZE):
            index_list = random.sample(range(n), BATCH_SIZE )
            inst_train_sentences = [train_sentences[idx] for idx in index_list]
            inst_train_insults = [train_insults[idx] for idx in index_list]
            inst_train_comment_length = [train_comment_length[idx] for idx in index_list]
            inst_train_comment_length_index = [train_comment_length_index[idx] for idx in index_list]
            inst_train_sentences_length = [train_sentences_length[idx] for idx in index_list]

            feed_dict = {sentences: inst_train_sentences, insult:inst_train_insults, \
                         comment_length: inst_train_comment_length, sentences_length: inst_train_sentences_length, keep_prob: 0.5,\
                         comment_length_index: inst_train_comment_length_index, learning_rate: learning_rate_ }
            
            
            _, current_loss = sess.run([opt_op, loss], feed_dict=feed_dict)
            total_loss += current_loss
     
        print(' Train loss:', total_loss / n, 'Time:', round((time.time()-t_0)/60,2),'minute') 

        train_predicted = []
        train_probability = []
        for i in range(split_num):
            train_feed_dict = {sentences: train_sentences_splited[i], insult: train_insults_splited[i],\
                               comment_length: train_comment_length_splited[i], sentences_length: train_sentences_length_splited[i],\
                               keep_prob: 1, comment_length_index: train_comment_length_index_splited[i]} 
            train_current_predicted, train_current_probability = sess.run([predict,probability], feed_dict=train_feed_dict)
            train_predicted = train_predicted + list(train_current_predicted)
            train_probability = train_probability + list(train_current_probability)

        train_f1 = f1_score(train_insults, train_predicted)
        train_auc = roc_auc_score(train_insults, np.array(train_probability)[:,1])
        print(' Train F1:', train_f1,' Train AUC:', train_auc)   

        test_predicted = []
        test_probability = []
        A = []
        for i in range(split_num):
            test_feed_dict = {sentences: test_sentences_splited[i], insult: test_insults_splited[i],\
                               comment_length: test_comment_length_splited[i], sentences_length: test_sentences_length_splited[i],\
                             keep_prob: 1, comment_length_index: test_comment_length_index_splited[i]} 
            test_current_predicted, test_current_probability = sess.run([predict,probability], feed_dict=test_feed_dict)
            a = sess.run(alphas, feed_dict=test_feed_dict)
            test_predicted = test_predicted + list(test_current_predicted)
            test_probability = test_probability + list(test_current_probability)
            A = A + list(a)

        test_f1 = f1_score(test_insults, test_predicted)
        test_auc = roc_auc_score(test_insults, np.array(test_probability)[:,1])
        print(' Dev F1:', test_f1,' Dev AUC:', test_auc)
        print(confusion_matrix(test_insults, test_predicted))

hyperparameters:  max_comment_len= 25 , max_sentence_len= 25 , sentence_hidden_size= 100 , comment_hidden_size= 100 , word2vec_size= 100
----- Epoch 1 -----
 Train loss: 0.00394458361908 Time: 0.97 minute
 Train F1: 0.750503597122  Train AUC: 0.917145438386
 Dev F1: 0.709883103082  Dev AUC: 0.80905108061
[[1021  137]
 [ 409  668]]
----- Epoch 2 -----
 Train loss: 0.00313132234801 Time: 0.96 minute
 Train F1: 0.741017964072  Train AUC: 0.926857484409
 Dev F1: 0.743988684583  Dev AUC: 0.823517879737
[[903 255]
 [288 789]]
----- Epoch 3 -----
 Train loss: 0.0028329289453 Time: 0.96 minute
 Train F1: 0.793719211823  Train AUC: 0.945047575869
 Dev F1: 0.687707641196  Dev AUC: 0.835913182367
[[1050  108]
 [ 456  621]]
----- Epoch 4 -----
 Train loss: 0.00263945960369 Time: 0.97 minute
 Train F1: 0.828494462736  Train AUC: 0.956180970504
 Dev F1: 0.706267029973  Dev AUC: 0.831994297471
[[1048  110]
 [ 429  648]]
----- Epoch 5 -----
 Train loss: 0.00228650387956 Time: 0.94 minute
 Train F1: 0.

In [ ]:
def plot_confusion_matrix_dict(matrix,rotation=45, outside_label=""):
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.colorbar()
    tick_marks = np.arange(0)
    plt.xticks(tick_marks, [0,1], rotation=rotation)
    plt.yticks(tick_marks, [0,1])
    
cm=confusion_matrix(test_insults, test_predicted)
print(cm)
plot_confusion_matrix_dict(cm)

In [282]:
print(predict_analysis(test_insults, test_predicted, np.array(test_probability)[:,1], 'fp', df_test)[50])
print('')
print(predict_analysis(test_insults, test_predicted, np.array(test_probability)[:,1], 'fn', df_test)[10])
print('')
print(predict_analysis(test_insults, test_predicted, np.array(test_probability)[:,1], 'tp', df_test)[50])

{' Who the hell are you to judge ?  Must be one of those who casts the first stone .  Your NWO will fail period .  You just do not know it yet ,  opps ,  now you do .  ': 0.55481035}

{' parker had more points ,  assists ,  rebounds then westbrook despite taking less shots .  How did Westbrook do a terrific job on parker ?  lol Thunder fans are not very bright ': 0.00075073628}

{' Non ho detto questo ,  quindi non inventare cose .  Anzi ,  ho detto che delle scie mi interessa poco e vanno verificate ,  quindi di che parliamo ?  Insunto ,  chi e contro il nucleare ,  esperti o meno che siano ,  sono tutti scemi ?  ': 0.0010902315}


* attention visualisation

In [416]:
from colored import fg, bg, attr


def attentionprint(comments_pd, A, k):
    attention = A[k]
    input_sentences = eval(comments_pd["Comment constructed"][k]) 
    print(input_sentences)
    print('Insult: ', comments_pd['Insult'][k])
    print('')
    for i,sentence in enumerate(input_sentences):
        if 0 <= attention[i] < 0.075 :
            print ('%s :' % (i+1),  '%s %s %s' % (fg(232), sentence, attr('bold')))
        else:
            if 0.075<= attention[i] < 0.15:
                color = 195
            elif 0.15<= attention[i] < 0.3:
                color = 159
            elif 0.3<= attention[i] < 0.475:
                color = 123
            elif 0.475<= attention[i] < 0.625:
                color = 51
            elif 0.625<= attention[i] < 0.75:
                color =14
            elif 0.75<= attention[i] < 0.875:  
                color = 81
            elif 0.875 <= attention[i] <= 1:
                color = 75
            print ('%s :' % (i+1),  '%s%s %s %s' % (fg(232), bg(color), sentence, attr('bold')))
    print('----------------------------------------------------------------------------------------------------------------------------------')
    print('')
k = 1097

print('----------------------------------------------------------------------------------------------------------------------------------')
print('')
attentionprint(df_test, A, 97)
attentionprint(df_test, A, 197)
attentionprint(df_test, A, 497)

----------------------------------------------------------------------------------------------------------------------------------

['isalill are you fucking Norwegian cow face', 'Fuck norway bitch', 'Moo moo']
Insult:  1

1 :  isalill are you fucking Norwegian cow face 
2 :  Fuck norway bitch 
3 :  Moo moo 
----------------------------------------------------------------------------------------------------------------------------------

['man anytime somebody makes a comment', 'especially one that accurately describes something', 'there is gotta be one motherfucker to come in an shake things up huh', 'shut your god damn mouth you fucking DIVA BOY']
Insult:  1

1 :  man anytime somebody makes a comment 
2 :  especially one that accurately describes something 
3 :  there is gotta be one motherfucker to come in an shake things up huh 
4 :  shut your god damn mouth you fucking DIVA BOY 
-------------------------------------------------------------------------------------------------------

In [ ]:
97,197,397,497